# Text Adventure Game

This notebook describes the foundations of a Text Adventure Game.

We will need two main classes:

1. `Node`
2. `Edge`

In addition, we'll also need a `Game` class to keep track of where we are.

## Nodes

The Node will keep track of where we are in a game. This might be a physical place, or a state.

For example, you might move from room to room in a castle. Those would be physical places.

Or, you might do something (like drink a potion) that would make you grow smaller. Those would be changing your state, or the state that you are in.

In [1]:
class Node():
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.edges = []
        
    def display(self):
        print(self.description)
        if self.edges:
            paths = ", ".join([edge.command for edge in self.edges])
            print("You see paths to the " + paths)

Here are a couple of nodes. You create them by giving a name, and a description:

In [2]:
start = Node("start", "You find yourself in front of a scary mansion.")
end = Node("end", "You completed the game!")

You can tell a node to display itself with: 

In [3]:
start.display()

You find yourself in front of a scary mansion.


In [4]:
end.display()

You completed the game!


## Edges

You can connect Nodes by creating an edge. An Edge takes a command (like "north") and the name of a Node to which that command would take you.

In [5]:
class Edge():
    def __init__(self, command, to_node_name):
        self.command = command
        self.to_node_name = to_node_name

For example, to make a command that takes you from "start" to "end" via the command "north", do this:

In [6]:
start.edges.append(Edge("north", "end"))

Now, when we display the start node again, we see:

In [7]:
start.display()

You find yourself in front of a scary mansion.
You see paths to the north


## The Text Adventure Game Engine

We call this an "engine" because it can really run any game of your making, and the code doesn't need to change. The Game simply takes a list of Nodes that already have their nodes set:

In [8]:
class Game():
    def __init__(self, nodes):
        self.nodes = {}
        self.current = None
        for node in nodes:
            self.nodes[node.name] = node
            if self.current is None:
                self.current = node
            
    def play(self):
        self.current = self.nodes["start"]
        while True:
            self.current.display()
            if self.current.edges:
                command = input("Enter a command: ")
                for node in self.current.edges:
                    if command == node.command:
                        self.current = self.nodes[node.to_node_name]
            else:
                break
            

To play this simple two-node game, you can:

In [9]:
game = Game([start, end])

Enter a command at the prompt:

In [10]:
game.play()

You find yourself in front of a scary mansion.
You see paths to the north


Enter a command:  north


You completed the game!


## Loading a Game from a File

To make it easy on us, the game designers, we will put all of the nodes and edges into a CSV (Comma-Separated Value) file:

In [11]:
%%file my_game.csv
"column1","column2","column3","column4"
"node","start","You find yourself in front of a scary mansion."
"node","end","You completed the game!"
"edge","start","north","end"

Overwriting my_game.csv


Each game will be saved on the harddrive with a name (here it is "my_game.csv").

This simple "spreadsheet" file has 4 columns:

1. The first column in each row is either "node" or "edge" indicating whether it is a node or edge type
2. The second depends on the type. If node type, then it is the name. If an edge type, then it is the name of the node it applies to.
3. The third column is the description, if a node row. Else, it is a command
4. The forth column is empty if it is a node row. It contains the name of the node to go to if it is an edge

We now will write a loading program to create a game.

We use the csv.DictReader class that reads the csv file into a dictionary.

In [12]:
import csv

def load(filename):
    with open(filename) as csvfile:
        nodes = {}
        fieldnames = ['column1', 'column2', 'column3', 'column4']
        game_reader = csv.DictReader(csvfile, delimiter=",", fieldnames=fieldnames)
        for line, row in enumerate(game_reader):
            if line > 0: # skip
                if row["column1"] == "node":
                    node = Node(row["column2"], row["column3"])
                    nodes[node.name] = node
                elif row["column1"] == "edge":
                    from_node_name = row['column2']
                    command = row['column3']
                    to_node_name = row['column4']
                    nodes[from_node_name].edges.append(Edge(command, to_node_name))
    return Game(nodes.values())

Now, instead of doing:

In [13]:
start = Node("start", "You find yourself in front of a scary mansion.")
end = Node("end", "You completed the game!")
start.edges.append(Edge("north", "end"))
game = Game([start, end])

In [14]:
import pandas as pd

pd.read_csv("my_game.csv")

,column1,column2,column3,column4
0,node,start,You find yourself in front of a scary mansion.,NaN
1,node,end,You completed the game!,NaN
2,edge,start,north,end


We can load that same information from a file:

In [15]:
game = load("my_game.csv")

In [16]:
game.play()

You find yourself in front of a scary mansion.
You see paths to the north


Enter a command:  north


You completed the game!
